In [1]:
######JM Gawrych  11/30/2018 rev 3/11/2019
#### Gets multiple Hail Vison geojson files from our featurizer catalog and combines them

#####B Snyder 6/29/2019 Added to Watson Studio
#!pip install arcgis #You must do this the first time you run the notebook!

import requests,json,datetime # need to install these libraries to run
from arcgis.gis import GIS
from arcgis import features
from arcgis.features import FeatureLayerCollection

##All TWC visual products have a product name and number
prod_num=str(335) # Radar Derived 24-Hour HailVision. 
prod_name='MESHMax1440min' # Per TWC naming conventions
APIKEY='YOUR_TWC_KEY_HERE'  #Enter your API key 


########################PRODUCT INVENTORY CALL################################  
def InvCall():
    base1 = 'https://api.weather.com/v2/tiler/info?products='+prod_num+':'+prod_name
    base2 = '&apiKey='+APIKEY
    url = base1 + base2 
    APIcall = requests.get(url) #Our API Call     to grab the latest forecast
    if APIcall.status_code == 200:  #if(APIcall.ok):
        data = APIcall.json() 
        for x in range(0,1):
            for y in range(0,1):
                timestamp = data["layers"][prod_num][prod_name]["dimensions"][x]["t"][y]
            return timestamp

##########################   API CALL   ###################################
def APIcall(timestamp,ts): # function for pulling data 
    count=0

    num_layers = 6  #10
    inc=12.7  
    fc = {
        "type": "FeatureCollection",
        "features": []
    }
   
    low_limit = 12.7   ##Thresholds are in mm, min is 5 mm (0.2") up to 50 mm (~2") 
    low_limitS = str(low_limit)
    high_limit=low_limit+12.7
    high_limitS = str(high_limit)      
                 
    base1 = 'https://api.weather.com/v2/featurizer/feature?product='+prod_num+':'+prod_name
    base2 = '&t='+timestamp + '&apiKey='+APIKEY+'&geometryType=polygon'
    
    for x in range(0,num_layers):
        
        count=count+1
        base3 = '&threshold='+low_limitS

        url = base1 + base2 + base3  # make API URL   
        print (url)
        data = requests.get(url).json() #Our API Call
        
        for y in data['features']:           
            f = {"type": "Feature", "properties": {}, "geometry": None}
            
            f['geometry'] = y['geometry']
            #convert to inches
            HailSizeIn= round(float(y['properties']['threshold'])*0.0394,1)
            f['properties']['HailSize'] = HailSizeIn  # y['properties']['threshold']
            f['properties']['Date'] = ts
            fc['features'].append(f)           
        
          
        low_limit=low_limit+inc
        low_limitS=str(low_limit)                  
        high_limit=high_limit+inc
        high_limitS=str(high_limit) 
    
    with open("HailVision_"+ts+".geojson", "w") as outfile:
        json.dump(fc, outfile)            
    with open("HailVision.geojson", "w") as outfile:
        json.dump(fc, outfile)       

################################################################################
timestamp = InvCall()  ## Grab time stamp

now=datetime.datetime.utcnow()
year=str(now.year)
month=str('{:02d}'.format(now.month))
day=str('{:02d}'.format(now.day))
hour=str('{:02d}'.format(now.hour))
ts=year+month+day+hour+'00'

APIcall(timestamp,ts)

################################################################################
##  ADD CONTENT TO ARCGIS  
# Connect to the GIS Using Your Credentials
portalURL = "http://ibmmain.maps.arcgis.com"
userID = "YOUR_ESRI_USERNAME"
password = "YOUR_ESRI_PASSWORD"
gis = GIS(portalURL, userID, password)

#Update each piece of content
existing_ID = '465ef129d4314343864b73f49d48c83e'  # Map ID
wx_item = gis.content.get(existing_ID)
collection = features.FeatureLayerCollection.fromitem(wx_item)
collection.manager.overwrite('HailVision.geojson')  
print ("Updating Layer")


    100% |████████████████████████████████| 1.4MB 15.8MB/s ta 0:00:01
    100% |████████████████████████████████| 10.0MB 3.5MB/s eta 0:00:01
    100% |████████████████████████████████| 194kB 38.6MB/s ta 0:00:01
    100% |████████████████████████████████| 16.6MB 2.0MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 29.0MB/s ta 0:00:01
  Running setup.py bdist_wheel for arcgis ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/27/98/41/70f5cde7b364e66f183b89e78693143d8ba297f0108571f82d
  Running setup.py bdist_wheel for pyshp ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/e0/25/e1/c02348b30bde207b78388f29df97bdabb0567056fdd3e209d8
Successfully built arcgis pyshp


tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
jupyterlab-server 1.0.0 has requirement jsonschema>=3.0.1, but you'll have jsonschema 2.6.0 which is incompatible.
  Found existing installation: pandas 0.21.0
    Uninstalling pandas-0.21.0:
      Successfully uninstalled pandas-0.21.0
https://api.weather.com/v2/featurizer/feature?product=335:MESHMax1440min&t=1561834543000&apiKey=f60f9ecb790749d38f9ecb790729d3be&geometryType=polygon&threshold=12.7
https://api.weather.com/v2/featurizer/feature?product=335:MESHMax1440min&t=1561834543000&apiKey=f60f9ecb790749d38f9ecb790729d3be&geometryType=polygon&threshold=25.4
https://api.weather.com/v2/featurizer/feature?product=335:MESHMax1440min&t=1561834543000&apiKey=f60f9ecb790749d38f9ecb790729d3be&geometryType=polygon&threshold=38.099999999999994
https://api.weather.com/v2/featurizer/feature?product=335:MESHMax1440min&t=1561834543000&apiKey=f60f9ecb790749d38f9ecb790729d3be&geometryType=polygon&threshold=50.8
h